In [ ]:
%%capture
!pip install -U bitsandbytes

import os
os.kill(os.getpid(), 9) #restart the session

In [ ]:
%%capture
!pip install alfworld[full]
!alfworld-download
!pip install colorama

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
from google.colab import userdata
api_key = userdata.get('HF_TOKEN')

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
def get_llama3_8b_response(model, tokenizer, messages):
    # input_ids = tokenizer.apply_chat_template(
    #     messages,
    #     add_generation_prompt=True,
    #     return_tensors="pt"
    # ).to(model.device)
    input_ids = tokenizer(messages, return_tensors="pt").to(DEVICE)

    # if input_ids.shape[1] >= 4048:
    #     messages = messages[-10:]
    #     input_ids = tokenizer.apply_chat_template(
    #         messages,
    #         add_generation_prompt=True,
    #         return_tensors="pt"
    #     ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=256,
            eos_token_id=terminators,
            do_sample=True,
            temperature=0.6,
            top_p=0.9,
        )
    response = outputs[0][input_ids.shape[-1]:]
    result = tokenizer.decode(response, skip_special_tokens=True)
    # print(result)
    return result

In [ ]:
def llm(model, tokenizer, message):
    # formatted_prompt +="\nAssistant: "
    inputs = tokenizer(message, return_tensors="pt", truncation=True).to(DEVICE)

    with torch.no_grad():
        output = model.generate(**inputs,
                                max_new_tokens=500,
                                temperature = 0.1,
                                top_p = 1.0,   # No nucleus sampling
                                do_sample = False,
                                repetition_penalty = 1.1)

    response_text = tokenizer.decode(output[0], skip_special_tokens = True).strip()

    # if "Assistant:" in response_text:
    #   response_text = response_text.split("Assistant:")[-1].strip()
    return response_text

In [ ]:
models_id = {
    'meta-llama/Llama-3.1-8B-Instruct': 'meta-llama/Llama-3.1-8B-Instruct',
    'AronXiang/RetrospexLLaMA3': 'AronXiang/RetrospexLLaMA3',
    'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B': 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
}

In [ ]:
from openai import OpenAI



client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1/",
    api_key= api_key
)

# MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"
# MODEL_NAME = "AronXiang/RetrospexLLaMA3"
# MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"


def chat_completion(prompt, stop=["\n"]):
    messages = [
        {
            "role": "user",
            "content": prompt
        }
    ]
    # model="google/gemma-2-2b-it"
    model = models_id['meta-llama/Llama-3.1-8B-Instruct']
    max_tokens=500
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        stream=False,
        stop=stop
    )

    # Extract the text from the response

    return response.choices[0].message.content

# Example usage:
prompt = "What is the square root of -1?"
result = chat_completion(prompt)
print(result)


BadRequestError: Error code: 400 - {'error': 'Model requires a Pro subscription; check out hf.co/pricing to learn more. Make sure to include your HF token in your query.'}

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = "meta-llama/Llama-3.1-8B-Instruct"


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enables 4-bit quantization
    bnb_4bit_quant_type="nf4",  # NormalFloat4 (NF4) for better precision
    bnb_4bit_compute_dtype=torch.float16,  # Compute in float16 (reduce memory usage)
    bnb_4bit_use_double_quant=True  # Double quantization for additional compression
)


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)


model.to(DEVICE)

print(f"✅ Model loaded successfully on {DEVICE} with 4-bit quantization!")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

✅ Model loaded successfully on cuda with 4-bit quantization!


In [ ]:
messages = [{'role':'user',
             'content':'What is your name?'}]
get_llama3_8b_response(model, tokenizer, messages)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"I'm an artificial intelligence model known as a conversational AI. I don't have a personal name, but you can think of me as a helpful assistant or a conversational partner. I'm here to assist you with any questions or topics you'd like to discuss."

In [ ]:
llm(model, tokenizer, 'how are you?')

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


"how are you? I am doing well, thanks for asking. I was wondering if you could help me with a problem I'm having with my car's engine.\nI've been experiencing some issues with my car's engine, and I'm not sure what's causing them. The symptoms include:\n* The engine is making a strange noise when it's running\n* The car is hesitating or stalling occasionally\n* The fuel efficiency has decreased significantly\n\nI've tried to troubleshoot the issue on my own, but I haven't been able to pinpoint the cause. I was hoping that you could offer some advice or guidance on how to diagnose the problem.\n\nCould you please provide some suggestions on where to start looking for the source of the issue?\n\nThank you so much for your time and assistance!\n\nHere are some possible causes of the problems you're experiencing:\n\n1.  **Faulty Spark Plugs**: Worn-out spark plugs can cause misfires, which may lead to hesitation or stalling. Check the condition of your spark plugs and consider replacing th

In [ ]:
import os
import sys
import json
import yaml
import importlib
import alfworld


In [ ]:
from typing import List, Dict


class EnvironmentHistory:
    def __init__(self, base_query: str, start_info, memory: List[str], history: List[Dict[str, str]] = []) -> None:
        self._cur_query: str = f'{_get_base_query(base_query, start_info, memory)}'
        self._history: List[Dict[str, str]] = history
        self._last_action: str = ''
        self._is_exhausted: bool = False

    def add(self, label: str, value: str) -> None:
        assert label in ['action', 'observation', 'human_edit']
        self._history += [{
            'label': label,
            'value': value,
        }]
        if label == 'action':
            if value == self._last_action:
                self._is_exhausted = True
            else:
                self._last_action = value

    def check_is_exhausted(self) -> bool:
        return self._is_exhausted

    def reset(self) -> None:
        self._history = []

    def __str__(self) -> str:
        s: str = self._cur_query + '\n'
        for i, item in enumerate(self._history):
            if item['label'] == 'action':
                s += f'> {item["value"]}'
            elif item['label'] == 'observation':
                s += item['value']
            # NOT CURRENTLY SUPPORTED
            elif item['label'] == 'human_edit':
                s += f'[human edit]: {item["value"]}'
            if i != len(self._history) - 1:
                s += '\n'
        return s

def _get_base_query(base_query: str, start_info: str, memory: List[str]) -> str:
    query = base_query

    # add memory if it exists
    if len(memory) > 0:
        query += '\n\nYour memory for the task below:'
        for i, m in enumerate(memory):
            query += f'\nTrial {i}:\n{m.strip()}'
    query += f"\nHere is the task:\n{start_info}"
    return query


In [ ]:
from typing import Literal

Model = Literal["gpt-4", "gpt-3.5-turbo", "meta-llama/Llama-3.1-8B-Instruct"]

In [ ]:
from typing import List, Dict, Any, Tuple

def process_ob(ob):
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]
    return ob

def alfworld_run(env, base_prompt, memory: List[str], to_print=True, ob='', model: Model = "text-davinci-003") -> Tuple[EnvironmentHistory, bool]:
    if len(memory) > 3:
        env_history = EnvironmentHistory(base_prompt, ob, memory[-3:], [])
    else:
        env_history = EnvironmentHistory(base_prompt, ob, memory, [])
    env_history.reset()
    if to_print:
        print(ob)
        sys.stdout.flush()
    cur_step = 0
    while cur_step < 49:
        action = llm(model=model, tokenizer= tokenizer, message = str(env_history) + ">").strip()
        env_history.add("action", action)
        observation, reward, done, info = env.step([action])
        observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
        if action.startswith('think:'):
            observation = 'OK.'
        env_history.add("observation", observation)
        if to_print:
            print(f'> {action}\n{observation}')
            sys.stdout.flush()
        if done:
            return env_history, True
        elif env_history.check_is_exhausted():
            return env_history, False
        cur_step += 1
    return env_history, False

PREFIXES = {
    'pick_and_place': 'put',
    'pick_clean_then_place': 'clean',
    'pick_heat_then_place': 'heat',
    'pick_cool_then_place': 'cool',
    'look_at_obj': 'examine',
    'pick_two_obj': 'puttwo'
}

def run_trial(
        trial_log_path: str,
        world_log_path: str,
        trial_idx: int,
        env_configs: List[Dict[str, Any]],
        use_memory: bool,
        model: Model,
    ) -> List[Dict[str, Any]]:
    importlib.reload(alfworld)
    importlib.reload(alfworld.agents.environment.alfred_tw_env)

    with open('base_config.yaml') as reader:
        config = yaml.safe_load(reader)
    split = "eval_out_of_distribution"

    # env = getattr(alfworld.agents.environment, config["env"]["type"])(config, train_eval=split)
    env = getattr(alfworld.agents.environment.alfred_tw_env, 'AlfredTWEnv')(config, train_eval=split)

    env = env.init_env(batch_size=1)

    num_successes: int = 0
    num_additional_successes: int = 0
    num_envs: int = len(env_configs)

    for z, env_config in enumerate(env_configs):
        ob, info = env.reset()
        ob = '\n'.join(ob[0].split('\n\n')[1:])
        name = '/'.join(info['extra.gamefile'][0].split('/')[-3:-1])

        print(f"using {name}")

        if env_config["is_success"]:
            num_successes += 1

            # log to world log
            with open(world_log_path, 'a') as wf:
                wf.write(f'Environment #{z} Trial #{trial_idx}: SUCCESS\n')
            with open(trial_log_path, 'a') as wf:
                wf.write(f'\n#####\n\nEnvironment #{z}: Success\n\n#####\n')
            continue

        for i, (k, v) in enumerate(PREFIXES.items()):
            if name.startswith(k):
                base_prompt = 'Interact with a household to solve a task. Here are two examples.\n' + d[f'react_{v}_1'] + d[f'react_{v}_0']
                final_env_history, is_success = alfworld_run(env, base_prompt, env_config["memory"] if use_memory else [], to_print=True, ob=ob, model=model)

                # update env config
                if is_success:
                    status_str: str = f'Environment #{z} Trial #{trial_idx}: SUCCESS'
                    env_configs[z]['is_success'] = True
                    num_successes += 1
                    num_additional_successes += 1
                else:
                    status_str: str = f'Environment #{z} Trial #{trial_idx}: FAIL'

                # log to world log
                with open(world_log_path, 'a') as f:
                    f.write(status_str + '\n')

                # log env results to trial log
                with open(trial_log_path, 'a') as wf:
                    wf.write(f'\n#####\n\nEnvironment #{z}:\n{str(final_env_history)}\n\nSTATUS: {"OK" if is_success else "FAIL"}\n\n#####\n')

    # close environment object
    env.close()

    # log trial results to trial and world logs
    log_str: str = f"""
-----
SUCCESS: {num_successes}
ADDITIONAL SUCCESS: {num_additional_successes}
FAIL: {num_envs - num_successes}
TOTAL: {num_envs}
ACCURACY: {round(num_successes / num_envs, 2)}
-----"""
    with open(trial_log_path, 'a') as wf:
        wf.write(log_str)
    with open(world_log_path, 'a') as wf:
        wf.write(log_str + '\n')

    return env_configs


In [ ]:
PROMPT_FILE = 'alfworld_3prompts.json'
with open(PROMPT_FILE, 'r') as f:
    d = json.load(f)

In [ ]:
num_envs = 134
num_trials = 10
trial_log_path = 'trial_log.log'
world_log_path = 'world.log'
trial_idx = 1
env_configs: List[Dict[str, Any]] = []
for i in range(num_envs):
    env_configs += [{
        'name': f'env_{i}',
        'memory': [],
        'is_success': False,
        'skip': False
    }]
# model = models_id['meta-llama/Llama-3.1-8B-Instruct']
use_memory = True
run_trial(trial_log_path, world_log_path, trial_idx, env_configs, use_memory, model)

Initializing AlfredTWEnv...


100%|██████████| 341/341 [00:00<00:00, 1502.92it/s]


Overall we have 134 games in split=eval_out_of_distribution
Evaluating with 134 games
using look_at_obj_in_light-AlarmClock-None-DeskLamp-308/trial_T20190908_222917_366542
You are in the middle of a room. Looking quickly around you, you see a bed 1, a desk 2, a desk 1, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a garbagecan 1, a laundryhamper 1, a safe 1, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.
Your task is to: examine the alarmclock with the desklamp.


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


> Interact with a household to solve a task. Here are two examples.
You are in the middle of a room. Looking quickly around you, you see a bed 1, a drawer 10, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a shelf 9, a shelf 8, a shelf 7, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.
Your task is to: examine the pen with the desklamp.
> think: To solve the task, I need to find and take a pen, then find and use a desklamp. First I need to find a pen. A pen is more likely to appear in drawer (1-10), shelf (1-9), bed (1), garbagecan (1). I can check one by one, starting with drawer 1.
OK.
> go to drawer 1
The drawer 1 is closed.
> open drawer 1
You open the drawer 1. The drawer 1 is open. In it, you see nothing.
> go to drawer 2
The drawer 2 is closed.
> open drawer 2
You open the drawer 2. The drawer 2 is open. In it, you see nothing.
> go to drawer 3
The drawer 3 is closed.


...

> Interact with a household to solve a task. Here are two examples.
You are in the middle of a room. Looking quickly around you, you see a bed 1, a drawer 10, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a shelf 9, a shelf 8, a shelf 7, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.
Your task is to: examine the pen with the desklamp.
> think: To solve the task, I need to find and take a pen, then find and use a desklamp. First I need to find a pen. A pen is more likely to appear in drawer (1-10), shelf (1-9), bed (1), garbagecan (1). I can check one by one, starting with drawer 1.
OK.
> go to drawer 1
The drawer 1 is closed.
> open drawer 1
You open the drawer 1. The drawer 1 is open. In it, you see nothing.
> go to drawer 2
The drawer 2 is closed.
> open drawer 2
You open the drawer 2. The drawer 2 is open. In it, you see nothing.
> go to drawer 3
The drawer 3 is closed.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


> Interact with a household to solve a task. Here are two examples.
You are in the middle of a room. Looking quickly around you, you see a bed 1, a drawer 10, a drawer 9, a drawer 8, a drawer 7, a drawer 6, a drawer 5, a drawer 4, a drawer 3, a drawer 2, a drawer 1, a dresser 1, a garbagecan 1, a shelf 9, a shelf 8, a shelf 7, a shelf 6, a shelf 5, a shelf 4, a shelf 3, a shelf 2, and a shelf 1.
Your task is to: examine the pen with the desklamp.
> think: To solve the task, I need to find and take a pen, then find and use a desklamp. First I need to find a pen. A pen is more likely to appear in drawer (1-10), shelf (1-9), bed (1), garbagecan (1). I can check one by one, starting with drawer 1.
OK.
> go to drawer 1
The drawer 1 is closed.
> open drawer 1
You open the drawer 1. The drawer 1 is open. In it, you see nothing.
> go to drawer 2
The drawer 2 is closed.
> open drawer 2
You open the drawer 2. The drawer 2 is open. In it, you see nothing.
> go to drawer 3
The drawer 3 is closed.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 850.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 210.12 MiB is free. Process 211225 has 14.53 GiB memory in use. Of the allocated memory 11.74 GiB is allocated by PyTorch, and 2.67 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import alfworld

In [ ]:
import alfworld.agents.environment


In [ ]:
import alfworld.agents.environment.alfred_tw_env


In [ ]:
!ls

alfworld_3prompts.json	base_config.yaml  sample_data


In [ ]:
import gc

In [ ]:
gc.collect()

479